<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [80]:
# scientific package imports
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
from matplotlib import rcParams

import astropy.units as u

from psycopg2.extras import execute_values

# import my own classes
# from galaxy.galaxy import Galaxy
# from galaxy.galaxies import Galaxies
# from galaxy.massprofile import MassProfile
from galaxy.timecourse import TimeCourse
# from galaxy.centerofmass import CenterOfMass
# from galaxy.plots import Plots
from galaxy.db import DB

In [2]:
db = DB()
tc = TimeCourse()
cur = db.get_cursor()

In [32]:
def get_distances(gal, snap):
    data = np.array(db.get_xyz(gal, snap),
                    dtype=[('pnum', '<i4'), ('x', '<f4'), ('y', '<f4'), ('z', '<f4')])
    xyz = np.array([data[xi] for xi in ('x','y','z')])
    MW_com, _ = tc.get_one_com('MW', snap)
    M31_com, _ = tc.get_one_com('M31', snap)
    M33_com, _ = tc.get_one_com('M33', snap)
    d_MW = norm(xyz - MW_com[:, np.newaxis], axis=0)
    d_M31 = norm(xyz - M31_com[:, np.newaxis], axis=0)
    d_M33 = norm(xyz - M33_com[:, np.newaxis], axis=0)
    return data['pnum'], d_MW, d_M31, d_M33

In [79]:
def write_dist_db_old(gal, snap):
    pnum, d_MW, d_M31, d_M33 = get_distances(gal, snap)
    dist = np.array([d_MW, d_M31, d_M33])
    nearest = np.argmin(dist, axis=0)


    colheads = ','.join(['galname','snap','pnum','d_mw','d_m31','d_m33','nearest'])
    query = f"""
        INSERT INTO distances( {colheads} ) 
        VALUES (%s,%s,%s,%s,%s,%s,%s)
        ON CONFLICT DO NOTHING
        """

    for i in range(len(d_MW)):
        rec = [gal, snap, int(pnum[i]), float(d_MW[i]), 
               float(d_M31[i]), float(d_M33[i]), float(nearest[i])]
        cur.execute(query, rec)
        if i%1000 == 0:
            print(i//1000, end=', ')

In [99]:
def write_dist_db(gal, snap):
    pnum, d_MW, d_M31, d_M33 = get_distances(gal, snap)
    dist = np.array([d_MW, d_M31, d_M33])
    nearest = np.argmin(dist, axis=0)
    
    data = [(gal, snap, int(pnum[i]), float(d_MW[i]), 
               float(d_M31[i]), float(d_M33[i]), int(nearest[i]))
                for i in range(len(pnum))]
#     return data

    colheads = ','.join(['galname','snap','pnum','d_mw','d_m31','d_m33','nearest'])
    query = f"""
        INSERT INTO distances( {colheads} ) 
        VALUES %s
        ON CONFLICT DO NOTHING
        """
    
    execute_values(cur, query, data, page_size=1000)

In [ ]:
for snap in range(10, 802, 10):
    for gal in ('MW','M31','M33'):
        print(gal, snap)
        data = write_dist_db(gal, snap)

MW 10
M31 10


In [103]:
?range

In [88]:
nearest = np.argmin(dist, axis=0)

In [89]:
np.save('MW_0_nearest', nearest)

In [60]:
index = np.where(nearest > 0)

In [61]:
index

(array([     3,      5,      8, ..., 670280, 673446, 673927]),)

In [63]:
len(index[0])

18035

In [64]:
data = np.array(db.get_xyz(gal, snap),
                    dtype=[('pnum', '<i4'), ('x', '<f4'), ('y', '<f4'), ('z', '<f4')])

In [65]:
data[index]

array([(     3, 2759.02  , 1955.83 , -1998.68  ),
       (     5, -983.576 , -490.894,  -316.738 ),
       (     8,  -33.2658,  884.657,  -311.281 ), ...,
       (670280, -661.786 ,  240.814,    20.1039),
       (673446, -460.166 ,  150.04 ,  -873.53  ),
       (673927,  -99.3205,  251.37 ,  -552.577 )],
      dtype=[('pnum', '<i4'), ('x', '<f4'), ('y', '<f4'), ('z', '<f4')])

In [66]:
d_MW[index]

array([3927.0854 , 1143.8733 ,  935.6398 , ...,  702.9106 ,  996.64685,
        612.7346 ], dtype=float32)

In [67]:
d_M31[index]

array([3818.949  , 1258.2881 ,  440.44144, ...,  557.62036,  752.66907,
        528.1677 ], dtype=float32)